In [47]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


In [48]:
assistant = client.beta.assistants.create(
    name='Math tutor',
    instructions='You are a personal math tutor. Write and run code to answer math questions.',
    tools=[{'type': 'code_interpreter'}],
    model='gpt-3.5-turbo',
)



In [49]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_hVCUAlrouysFazCHv53r0Ov0', created_at=1720050112, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [50]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'Solve this problem: 3x + 11 = 14'
)

In [51]:
print(message)

Message(id='msg_URxGSf8m7KIdrCI4ASsIxhCJ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1720050112, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_hVCUAlrouysFazCHv53r0Ov0')


In [52]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    
    assistant_id= assistant.id,
)

In [53]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [54]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_URxGSf8m7KIdrCI4ASsIxhCJ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1720050112, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_hVCUAlrouysFazCHv53r0Ov0')], object='list', first_id='msg_URxGSf8m7KIdrCI4ASsIxhCJ', last_id='msg_URxGSf8m7KIdrCI4ASsIxhCJ', has_more=False)


In [55]:
for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14


In [56]:
#step 1

assistant = client.beta.assistants.create(
  name="Machine Learning Researcher",
  instructions="'You are a machine learning researcher, answer questions on the research paper.'",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"}],
)

In [57]:
#step 2

vector_store = client.beta.vector_stores.create(name='memgpt-paper')
file_paths = ['./memgpt-paper.pdf']
file_streams = [open(file_path, 'rb') for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams,
)
print(file_batch.status)
print(file_batch.file_counts)
print(file_batch)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
VectorStoreFileBatch(id='vsfb_d6b3d54639b944c895a3e407e716d54a', created_at=1720050116, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), object='vector_store.file_batch', status='completed', vector_store_id='vs_gZiCiZUhQWUALQAOcYPE6dJZ')


In [58]:
# step 3

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)



In [59]:
# step 4

# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("memgpt-paper.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Summarize the research paper.",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_SAROVR96NNpZcTFZ3ArtiktD'])


In [60]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

print(messages)
message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))


[Message(id='msg_I4u1m8na2dPShhiE9AkxDNap', assistant_id='asst_04ljt2Kh046vfq84Nj6UNXe5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=796, file_citation=FileCitation(file_id='file-h728z1UUQSHuGQyJ46FQDD2n'), start_index=784, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1555, file_citation=FileCitation(file_id='file-h728z1UUQSHuGQyJ46FQDD2n'), start_index=1543, text='【4:2†source】', type='file_citation')], value='The research paper titled "MemGPT: Towards LLMs as Operating Systems" introduces MemGPT, a system designed to address the limitations of Large Language Models (LLMs) due to their constrained context windows. The paper proposes virtual context management inspired by hierarchical memory systems in traditional operating systems to extend the context available to LLMs. MemGPT intelligently manages different storage tiers to effectively provide extended context within the LLM\'s l

In [61]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'How does memgpt allow LLMs to have unlimited context length?'
)

In [62]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

MemGPT allows Large Language Models (LLMs) to have an unlimited context length by implementing a virtual context management technique inspired by hierarchical memory systems in traditional operating systems. This technique provides the illusion of an extended virtual memory through paging between physical memory and disk. With MemGPT, different storage tiers are intelligently managed to effectively provide extended context within the LLM's limited context window. By allowing the LLM to manage what is placed in its own context via an 'LLM OS' like MemGPT, relevant historical data can be retrieved beyond the in-context data, and less relevant data can be evicted into external storage systems. This system combines a memory-hierarchy, OS functions, and event-based control flow to enable handling unbounded context using LLMs with finite context windows[0].
[0] memgpt-paper.pdf
